In [1]:
import numpy as np
import pandas as pd

In [2]:
#data = pd.read_csv("file3.csv") 
csv_1 = pd.read_csv(r"C:\Users\Admin\ablation study - 20 neurons\with attention and dimensionality reduction\lgcabnormalresult.csv") 
  
# changing cols with rename() 
new_data = csv_1.rename(columns = {0: "Feature1",
                                  1: "Feature2", 
                                  2: "Feature3", 
                                  3: "Feature4",
                                  4: "Feature5",
                                  5: "Feature6",
                                  6: "Feature7",
                                  7: "Feature8",
                                  8: "Feature9",
                                  9: "Feature10",
                                  10: "Feature11",
                                  11: "Feature12",
                                  12: "Feature13",
                                  13: "Feature14",
                                  14: "Feature15",
                                  15: "Feature16",
                                  16: "Feature17",
                                  17: "Feature18",
                                  18: "Feature19", 
                                  19: "Feature20", 
                                  20: "Label"}) 
  
# changing columns using .columns() 
csv_1.columns = ['Feature1', 'Feature2', 'Feature3', 'Feature4', 
                'Feature5', 'Feature6', 'Feature7', 'Feature8',
                 'Feature9', 'Feature10', 'Feature11', 'Feature12',
                 'Feature13', 'Feature14', 'Feature15', 'Feature16',
                 'Feature17', 'Feature18', 'Feature19', 'Feature20',"Label"] 
  

csv_1.to_csv("lgcabnormalresult.csv", index=None)

#https://datacarpentry.org/python-ecology-lesson/03-index-slice-subset/index.html
csv_1=csv_1[0:100]
csv_2=csv_2[0:40000]
merged = csv_1.merge(csv_2,on="Label")

merged.to_csv("attentiongan.csv", index=False)
#result = pd.concat([csv_1, csv_2], axis=1)
out = csv_1.append(csv_2)
print(out)
out.to_csv("attentiongan.csv", index=False)

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations


import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("lgcabnormalresult.csv",sep=",", index_col=None)

In [5]:
print (df.dtypes)

Feature1     float64
Feature2     float64
Feature3     float64
Feature4     float64
Feature5     float64
Feature6     float64
Feature7     float64
Feature8     float64
Feature9     float64
Feature10    float64
Feature11    float64
Feature12    float64
Feature13    float64
Feature14    float64
Feature15    float64
Feature16    float64
Feature17    float64
Feature18    float64
Feature19    float64
Feature20    float64
Label          int64
dtype: object


In [6]:
df.head

<bound method NDFrame.head of        Feature1  Feature2  Feature3  Feature4  Feature5  Feature6  Feature7  \
0           0.0       0.0       0.0       0.0       0.0  0.117106       0.0   
1           0.0       0.0       0.0       0.0       0.0  0.000000       0.0   
2           0.0       0.0       0.0       0.0       0.0  0.000000       0.0   
3           0.0       0.0       0.0       0.0       0.0  0.000000       0.0   
4           0.0       0.0       0.0       0.0       0.0  0.000000       0.0   
...         ...       ...       ...       ...       ...       ...       ...   
39994       0.0       0.0       0.0       0.0       0.0  0.000000       0.0   
39995       0.0       0.0       0.0       0.0       0.0  0.117106       0.0   
39996       0.0       0.0       0.0       0.0       0.0  0.117106       0.0   
39997       0.0       0.0       0.0       0.0       0.0  0.000000       0.0   
39998       0.0       0.0       0.0       0.0       0.0  0.000000       0.0   

       Feature8  Feat

In [7]:
#X = mydataframe.drop(['acol','bcol'], axis=1).values 
#y = mydataframe['targetvalue'].values
df_label=df.Label
X=df.drop(['Label'], axis=1).values 
y=df['Label'].values

In [8]:
df_label.to_csv('abnormallabel1.csv', index=False)

In [9]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(X,y,train_size=0.70, random_state=2)
print(len(x_train), "Training sequences",x_train.shape)
print(len(x_val), "Validation sequences",x_val.shape)
print(len(y_train), "Training sequences",y_train.shape)

27999 Training sequences (27999, 20)
12000 Validation sequences (12000, 20)
27999 Training sequences (27999,)


# Simple Attention
#https://www.analyticsvidhya.com/blog/2019/11/comprehensive-guide-attention-mechanism-deep-learning/

In [10]:
def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

In [11]:
def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

In [12]:
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

In [13]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Flatten, Activation,RepeatVector,Permute 
from tensorflow.keras.layers import TimeDistributed
maxlen=20
#vocab_size = x_train.shape[0]
vocab_size = 500000
visible = layers.Input(shape=(maxlen,))

embed=Embedding(vocab_size,20)(visible)

activations= keras.layers.GRU(20, return_sequences=True)(embed)

attention = TimeDistributed(Dense(1, activation='tanh'))(activations) 
attention = Flatten()(attention)
attention = Activation('softmax', name='attention_vec')(attention)
#attention = RepeatVector(1)(attention)
#attention = Permute([2, 1])(attention) 

#sent_representation = keras.layers.multiply([activations, attention])
#sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)
predictions=Dense(1, activation='sigmoid')(attention)

model = keras.Model(inputs=visible, outputs=predictions)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 20)            10000000  
_________________________________________________________________
gru (GRU)                    (None, 20, 20)            2520      
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 1)             21        
_________________________________________________________________
flatten (Flatten)            (None, 20)                0         
_________________________________________________________________
attention_vec (Activation)   (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21    

In [16]:
import time, datetime

start = datetime.datetime.now()
time.sleep(10)

In [17]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
history =model.fit(x_train, y_train, batch_size=1024, epochs=5, validation_data=(x_val, y_val))
#history =model.fit_generator(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Train on 27999 samples, validate on 12000 samples
Epoch 1/5
27999/27999 [==============================] - 16s 570us/sample - loss: 0.6884 - accuracy: 0.6708 - val_loss: 0.6732 - val_accuracy: 1.0000
Epoch 2/5
27999/27999 [==============================] - 9s 312us/sample - loss: 0.6586 - accuracy: 1.0000 - val_loss: 0.6397 - val_accuracy: 1.0000
Epoch 3/5
27999/27999 [==============================] - 8s 287us/sample - loss: 0.6137 - accuracy: 1.0000 - val_loss: 0.5826 - val_accuracy: 1.0000
Epoch 4/5
27999/27999 [==============================] - 9s 321us/sample - loss: 0.5642 - accuracy: 1.0000 - val_loss: 0.5467 - val_accuracy: 1.0000
Epoch 5/5
27999/27999 [==============================] - 10s 353us/sample - loss: 0.5274 - accuracy: 1.0000 - val_loss: 0.5092 - val_accuracy: 1.0000


In [18]:
end = datetime.datetime.now()

diff = (end - start)

datetime.timedelta(seconds=10, microseconds=885206)

diff_seconds = int(diff.total_seconds())

minute_seconds, seconds = divmod(diff_seconds, 60)
hours, minutes = divmod(minute_seconds, 60)
hms = f"{hours}h {minutes}m {seconds}s"

'0h 0m 10s'
print(hms) 

0h 1m 1s


In [19]:
model = keras.Model(inputs=model.input,
              outputs=[model.output, model.get_layer('attention_vec').output])

In [20]:
layer = model.get_layer('attention_vec')

In [21]:
outputs = []
keras_function = K.function([model.input], [layer.output])
outputs.append(keras_function([x_train, 0]))
print(outputs)

[[array([[0.10036256, 0.11093968, 0.11125921, ..., 0.02743869, 0.07859769,
        0.08771991],
       [0.10036256, 0.11093968, 0.11125921, ..., 0.02743869, 0.07859769,
        0.08771991],
       [0.10036256, 0.11093968, 0.11125921, ..., 0.02743869, 0.07859769,
        0.08771991],
       ...,
       [0.10036256, 0.11093968, 0.11125921, ..., 0.0274387 , 0.0785977 ,
        0.08771992],
       [0.10036256, 0.11093968, 0.11125921, ..., 0.0274387 , 0.0785977 ,
        0.08771992],
       [0.10036256, 0.11093968, 0.11125921, ..., 0.0274387 , 0.0785977 ,
        0.08771992]], dtype=float32)]]


In [22]:
import numpy as np
import keras
hidden_layers = keras.backend.function(
[layer.input],  # we will feed the function with the input of the first layer  
[layer.output,] # we want to get the output of the first layer
)
h=hidden_layers([X])
print(h)
print(np.shape(h))

Using TensorFlow backend.


[array([[0.04880884, 0.04880884, 0.04880884, ..., 0.04880884, 0.04880884,
        0.04880884],
       [0.04967906, 0.04967906, 0.04967906, ..., 0.04967906, 0.04967906,
        0.04967906],
       [0.04976183, 0.04976183, 0.04976183, ..., 0.04976183, 0.04976183,
        0.04976183],
       ...,
       [0.04880884, 0.04880884, 0.04880884, ..., 0.04880884, 0.04880884,
        0.04880884],
       [0.04960391, 0.04960391, 0.04960391, ..., 0.04960391, 0.04960391,
        0.04960391],
       [0.04981558, 0.04981558, 0.04981558, ..., 0.04981558, 0.04981558,
        0.04981558]], dtype=float32)]
(1, 39999, 20)


In [23]:
df = pd.DataFrame(np.concatenate(h))

In [24]:
df.to_csv('abnormalattentionvector.csv', index=False)

In [25]:
csv_1 = pd.read_csv('abnormalattentionvector.csv')
csv_2 = pd.read_csv('abnormallabel1.csv')

result = pd.concat([csv_1, csv_2], axis=1)

In [26]:
result.to_csv("attentionabnormalresult.csv", index=None, header=None)